In [1]:
# Mount Google Drive (required if running on Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Install required modules
# !pip install '/content/drive/Team Drives/Deep Learning Team Drive/yelp_dataset_challenge-master/yelp_util'
!pip install Cython
# !pip install word2vec
!pip install gensim
!pip install unidecode
!pip install textblob
!pip install wordcloud


# Import required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# import yelp_util
# from yelp_util import downloader as dl
import os
from scipy import misc
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
##Chinese REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewChin = pd.read_pickle(os.path.join(download_path, 'chinese_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=12345).permutation(reviewChin.shape[0])

# Preprocess
reviewChin = reviewChin.iloc[shuffidx].reset_index(drop=True)
reviewChin = reviewChin.loc[:, ['stars','text']]
reviewChin = reviewChin[reviewChin.stars != 3]
reviewChin.stars.value_counts() # check that there are no "3" stars
reviewChin['sentiment']=['pos' if (x>3) else 'neg' for x in reviewChin['stars']] #add the sentiment column

#lowercase/remove punctuations

import re

reviewChin['text']= [x.lower() for x in reviewChin['text']]
reviewChin['text'] = reviewChin['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewChin.head()

,stars,text,sentiment
0,2,my wife and i are big fans of the pei wei chai...,neg
1,4,great delivery and even better dinein experien...,pos
3,5,quality was excellent food was fresh and reaso...,pos
4,2,service was mediocre asked for the dishes to ...,neg
5,5,really enjoyed lunch here greeted by beautiful...,pos


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, CuDNNLSTM, Bidirectional
from keras.metrics import categorical_accuracy
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',                                  
                      lower=True,split=' ')
tokenizer.fit_on_texts(reviewChin['text'].values)

In [27]:
X = tokenizer.texts_to_sequences(reviewChin['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewChin['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(34413, 870) (34413, 2)
(8604, 870) (8604, 2)


In [24]:
##Italian REVIEWS##

# Download and unpickle data

reviewItaly = pd.read_pickle(os.path.join(download_path, 'italian_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=12345).permutation(reviewItaly.shape[0])

# Reset index so it stays in the new order
reviewItaly = reviewItaly.iloc[shuffidx].reset_index(drop=True)

#preprocess
reviewItaly = reviewItaly.loc[:, ['stars','text']]
reviewItaly = reviewItaly[reviewItaly.stars != 3]
reviewItaly.stars.value_counts() # check that there are no "3" stars
reviewItaly['sentiment']=['pos' if (x>3) else 'neg' for x in reviewItaly['stars']] #add the sentiment column

#remove punctuations
import re
reviewItaly['text']= [x.lower() for x in reviewItaly['text']]
reviewItaly['text'] = reviewItaly['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

#tokenizer
tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',                                  
                      lower=True,split=' ')
tokenizer.fit_on_texts(reviewItaly['text'].values)
XI = tokenizer.texts_to_sequences(reviewItaly['text'].values)
XI = pad_sequences(XI)

YI = pd.get_dummies(reviewItaly['sentiment']).values
X_trainI, X_testI, Y_trainI, Y_testI = train_test_split(XI,YI, test_size = 0.2, random_state = 22)
print(X_trainI.shape,Y_trainI.shape)
print(X_testI.shape,Y_testI.shape)


(57500, 932) (57500, 2)
(14375, 932) (14375, 2)


In [12]:
##MEXICAN RESTAURANT REVIEWS##

# Download and unpickle data

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'

reviewMex = pd.read_pickle(os.path.join(download_path, 'mexican_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewMex.shape[0])

# Reset index so it stays in the new order
reviewMex = reviewMex.iloc[shuffidx].reset_index(drop=True)

#preprocess
reviewMex = reviewMex.loc[:, ['stars','text']]
reviewMex = reviewMex[reviewMex.stars != 3]
reviewMex.stars.value_counts() # check that there are no "3" stars
reviewMex['sentiment']=['pos' if (x>3) else 'neg' for x in reviewMex['stars']] #add the sentiment column

#remove punctuations
import re
reviewMex['text']= [x.lower() for x in reviewMex['text']]
reviewMex['text'] = reviewMex['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

#tokenizer
tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',                                  
                      lower=True,split=' ')
tokenizer.fit_on_texts(reviewMex['text'].values)
XM = tokenizer.texts_to_sequences(reviewMex['text'].values)
XM = pad_sequences(XM)

YI = pd.get_dummies(reviewMex['sentiment']).values
X_trainM, X_testM, Y_trainM, Y_testM = train_test_split(XM,YM, test_size = 0.2, random_state = 42)
print(X_trainM.shape,Y_trainM.shape)
print(X_testM.shape,Y_testM.shape)


(64582, 913) (64582, 2)
(16146, 913) (16146, 2)


In [31]:
##HAIR SALON REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewHair = pd.read_pickle(os.path.join(download_path, 'salon_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=1234).permutation(reviewHair.shape[0])

# Preprocess
reviewHair = reviewHair.iloc[shuffidx].reset_index(drop=True)
reviewHair = reviewHair.loc[:, ['stars','text']]
reviewHair = reviewHair[reviewHair.stars != 3]
reviewHair.stars.value_counts() # check that there are no "3" stars
reviewHair['sentiment']=['pos' if (x>3) else 'neg' for x in reviewHair['stars']] #add the sentiment column

#lowercase/remove punctuations

import re

reviewHair['text']= [x.lower() for x in reviewHair['text']]
reviewHair['text'] = reviewHair['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

tokenizer.fit_on_texts(reviewHair['text'].values)
XH = tokenizer.texts_to_sequences(reviewHair['text'].values)
XH = pad_sequences(XH)
YH = pd.get_dummies(reviewHair['sentiment']).values
X_trainH, X_testH, Y_trainH, Y_testH = train_test_split(XH,YH, test_size = 0.2, random_state = 42)
print(X_trainH.shape,Y_trainH.shape)
print(X_testH.shape,Y_testH.shape)

(13218, 935) (13218, 2)
(3305, 935) (3305, 2)


In [35]:
##Home and Garden REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewHome = pd.read_pickle(os.path.join(download_path, 'homegarden_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=1234).permutation(reviewHome.shape[0])

# Preprocess
reviewHome = reviewHome.iloc[shuffidx].reset_index(drop=True)
reviewHome = reviewHome.loc[:, ['stars','text']]
reviewHome = reviewHome[reviewHome.stars != 3]
reviewHome.stars.value_counts() # check that there are no "3" stars
reviewHome['sentiment']=['pos' if (x>3) else 'neg' for x in reviewHome['stars']] #add the sentiment column

#lowercase/remove punctuations

import re

reviewHome['text']= [x.lower() for x in reviewHome['text']]
reviewHome['text'] = reviewHome['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

tokenizer.fit_on_texts(reviewHome['text'].values)
XHG = tokenizer.texts_to_sequences(reviewHome['text'].values)
XHG = pad_sequences(XHG)
YHG = pd.get_dummies(reviewHome['sentiment']).values
X_trainHG, X_testHG, Y_trainHG, Y_testHG = train_test_split(XHG,YHG, test_size = 0.2, random_state = 12)
print(X_trainHG.shape,Y_trainHG.shape)
print(X_testHG.shape,Y_testHG.shape)

(7288, 909) (7288, 2)
(1823, 909) (1823, 2)


In [39]:
##Band and Credit Union REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewBCU = pd.read_pickle(os.path.join(download_path, 'bankandcu_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=1234).permutation(reviewBCU.shape[0])

# Preprocess
reviewBCU = reviewBCU.iloc[shuffidx].reset_index(drop=True)
reviewBCU = reviewBCU.loc[:, ['stars','text']]
reviewBCU = reviewBCU[reviewBCU.stars != 3]
reviewBCU.stars.value_counts() # check that there are no "3" stars
reviewBCU['sentiment']=['pos' if (x>3) else 'neg' for x in reviewBCU['stars']] #add the sentiment column

#lowercase/remove punctuations

import re

reviewBCU['text']= [x.lower() for x in reviewBCU['text']]
reviewBCU['text'] = reviewBCU['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

tokenizer.fit_on_texts(reviewBCU['text'].values)
XBCU = tokenizer.texts_to_sequences(reviewBCU['text'].values)
XBCU = pad_sequences(XBCU)
YBCU = pd.get_dummies(reviewBCU['sentiment']).values
X_trainBCU, X_testBCU, Y_trainBCU, Y_testBCU = train_test_split(XBCU,YBCU, test_size = 0.2, random_state = 12)
print(X_trainBCU.shape,Y_trainBCU.shape)
print(X_testBCU.shape,Y_testBCU.shape)

(1071, 846) (1071, 2)
(268, 846) (268, 2)


In [7]:
###GENERAL LSTM###

embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
#model.add(Embedding(2000, embed_dim,input_length = XI.shape[1]))
model.add(Embedding(2000, embed_dim))
model.add(SpatialDropout1D(0.2))
#model.add(CuDNNLSTM(lstm_out, return_sequences=False, input_shape=(932, 128)))
model.add(CuDNNLSTM(lstm_out, return_sequences=False))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 128)         0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 200)               264000    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 520,402
Trainable params: 520,402
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
#Fit model (train on Chinese)
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 38715 samples, validate on 4302 samples
Epoch 1/2
 - 209s - loss: 0.1276 - acc: 0.9505 - val_loss: 0.1466 - val_acc: 0.9449
Epoch 2/2
 - 209s - loss: 0.1069 - acc: 0.9586 - val_loss: 0.1473 - val_acc: 0.9386


In [28]:
#Test on the Chinese restaurants
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.16
Validation Accuracy: 0.9378


In [29]:
#Test on the Italy set
score,acc = model.evaluate(X_testI, Y_testI, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.92
Validation Accuracy: 0.6216


In [16]:
#Test on the Mex set
score,acc = model.evaluate(X_testM, Y_testM, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.06
Validation Accuracy: 0.5741


In [32]:
#Test on the Hair Salon
score,acc = model.evaluate(X_testH, Y_testH, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.88
Validation Accuracy: 0.6539


In [36]:
#Test on the Home Garden
score,acc = model.evaluate(X_testHG, Y_testHG, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.04
Validation Accuracy: 0.6248


In [40]:
#Test on the BCU
score,acc = model.evaluate(X_testBCU, Y_testBCU, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.68
Validation Accuracy: 0.4142


In [41]:
### Try Bidirectional LSTM ###
### The following sections ###
### were exectued with the following hyper parameters ###

from keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D

embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(CuDNNLSTM(lstm_out, return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(lstm_out)))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, None, 128)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 400)         528000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 400)               963200    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 802       
Total params: 1,748,002
Trainable params: 1,748,002
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
#Fit model (train on Chinese)
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 38715 samples, validate on 4302 samples
Epoch 1/2
 - 855s - loss: 0.3657 - acc: 0.8328 - val_loss: 0.5115 - val_acc: 0.7053
Epoch 2/2
 - 855s - loss: 0.2680 - acc: 0.8862 - val_loss: 0.1614 - val_acc: 0.9358


In [43]:
#Test on the Chinese restaurants
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.15
Validation Accuracy: 0.9432


In [44]:
#Test on the Italy set
score,acc = model.evaluate(X_testI, Y_testI, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.93
Validation Accuracy: 0.6769


In [45]:
#Test on the Mex set
score,acc = model.evaluate(X_testM, Y_testM, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.10
Validation Accuracy: 0.6195


In [46]:
#Test on the Hair Salon
score,acc = model.evaluate(X_testH, Y_testH, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.90
Validation Accuracy: 0.6859


In [47]:
#Test on the Home Garden
score,acc = model.evaluate(X_testHG, Y_testHG, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.16
Validation Accuracy: 0.6336


In [48]:
#Test on the BCU
score,acc = model.evaluate(X_testBCU, Y_testBCU, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 1.78
Validation Accuracy: 0.4291
